In [2]:
!pip install sktime==0.7.0

In [4]:
from sktime.transformations.panel.reduce import Tabularizer
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from numpy.random import default_rng
from sktime.datasets import load_arrow_head
from sktime.utils.data_io import load_from_tsfile_to_dataframe
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sktime.utils.slope_and_trend import _slope
from sklearn.tree import DecisionTreeClassifier
from sktime.classification.compose import ComposableTimeSeriesForestClassifier
import time
import datetime
from sklearn.metrics import confusion_matrix
from sktime.utils.data_processing import from_nested_to_2d_array # sktime -0.7.0
# from sktime.datatypes._panel._convert import from_nested_to_2d_array 

In [5]:
ts_file_name = 'coswara-4k.ts'

def load_ts_file(ts_file_name):
    result = load_from_tsfile_to_dataframe(ts_file_name)
    X = pd.DataFrame(dtype="object")
    y = pd.Series(dtype="object")
    X = pd.concat([X, pd.DataFrame(result[0])])
    y = pd.concat([y, pd.Series(result[1])])
    return X, y

# 18s --> 9s
# virufy data 2m 18s, 1m 52s
# virufy data 17s (4000 Hz)
# coswara 2m 41s
X, y = load_ts_file(ts_file_name)

In [6]:
print(type(X))
print(X.info())
print(X.shape)
print(type(y))
print(y.shape)
print(y)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4800 entries, 0 to 4799
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   dim_0   4800 non-null   object
dtypes: object(1)
memory usage: 37.6+ KB
None
(4800, 1)
<class 'pandas.core.series.Series'>
(4800,)
0       breathing
1           cough
2        counting
3           vowel
4       breathing
          ...    
4795        vowel
4796    breathing
4797        cough
4798     counting
4799        vowel
Length: 4800, dtype: object


In [7]:
# 1m 7s
import numpy as np
from sklearn.linear_model import RidgeClassifierCV
from sklearn.pipeline import make_pipeline
# from sktime.datasets import load_arrow_head  # univariate dataset
# from sktime.datasets import load_basic_motions  # multivariate dataset
from sktime.transformations.panel.rocket import Rocket

In [8]:
# initialize rocket and transform the training data
# rocket = Rocket()  # by default, ROCKET uses 10,000 kernels
rocket = Rocket(normalise=True, random_state=42)  # by default, ROCKET uses 10,000 kernels

start = time.time()
rocket.fit(X)

end = time.time()
print(f'time={end - start:.5f} s')
print(f'time={str(datetime.timedelta(seconds=end-start))}')

# time=0.40729 s
# time=0:00:00.407293

time=1.45053 s
time=0:00:01.450530


In [9]:
# 02h 10m 53s completed
# virufy 48000 Hz, over 3h 11m
# virufy  4000 Hz, 26m 27s
# coswara 40 sets 2m 18s
start = time.time()
X_transform = rocket.transform(X)

end = time.time()
print(f'"rocket.transform" time={end - start:.5f} s')
print(f'time={str(datetime.timedelta(seconds=end-start))}')

#"rocket.transform" time=1830.83486 s
# time=0:30:30.834859

# "rocket.transform" time=1859.84968 s
# time=0:30:59.849679

# "rocket.transform" time=1922.48763 s
# time=0:32:02.487631

# "rocket.transform" time=1570.55321 s
# time=0:26:10.553214

# virufy 4000 Hz
# "rocket.transform" time=1587.36839 s
# time=0:26:27.368386

"rocket.transform" time=18829.78888 s
time=5:13:49.788878


In [10]:
print(type(X_transform))
print(X_transform.info())

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4800 entries, 0 to 4799
Columns: 20000 entries, 0 to 19999
dtypes: float64(20000)
memory usage: 732.4 MB
None


In [11]:
# save to file
# 2m 43s completed
start = time.time()
X_transform.to_csv('coswara_rocket_X_transform.csv', index=False)
end = time.time()
print(f'time={end - start:.5f} s')
print(f'time={str(datetime.timedelta(seconds=end-start))}')

time=202.94070 s
time=0:03:22.940696


In [12]:
X_transform_2 = pd.read_csv('coswara_rocket_X_transform.csv')
print(X_transform_2.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4800 entries, 0 to 4799
Columns: 20000 entries, 0 to 19999
dtypes: float64(20000)
memory usage: 732.4 MB
None
